# Binary Search Trees (BST)
Yang Xi<br>
29 Sep 2020

<br>

* Introduction
* BST Implementation
    * BST Operations
* References

<br>

## Introduction

Binary search on a list and hash tables are two implementations of **map** abstract data type (ADT). **Binary Search Trees (BST)** is another way to map a key to a value.

BST provides efficient searching through **BST property**
* keys less than the parent are in the left subtree
* keys greater than the parent are in the right subtree

Example:
![](images/binary_search_tree_example_1.jpg)


# BST Implementation
The `TreeNode` class is the basic building block of BST.

* `__iter__` allows iternation through the tree.
    * See how it is implemented with a generator!

In [1]:
class TreeNode:
    
    def __init__(self,key,val,left=None,right=None,parent=None):
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent

    def hasLeftChild(self):
        return self.leftChild

    def hasRightChild(self):
        return self.rightChild

    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self

    def isRightChild(self):
        return self.parent and self.parent.rightChild == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.rightChild or self.leftChild)

    def hasAnyChildren(self):
        return self.rightChild or self.leftChild

    def hasBothChildren(self):
        return self.rightChild and self.leftChild

    def replaceNodeData(self,key,value,lc,rc):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self
            
    def __iter__(self): # inorder traversal
        if self:
            if self.hasLeftChild():
                for elem in self.leftChild:
                    yield elem
            yield self.key
            if self.hasRightChild():
                for elem in self.rightChild:
                    yield elem

### BST Operations
* **put()** add a node to the tree.<br>
It first checks if the tree already has a root.
    * If not, create a new `TreeNode` and install it as the root.
    * If yes, recursively call `_put()` to find the position where the new node should be installed.

**Note**: this implementation does not handle duplicated keys properly, which is placed under the right subtree of the original key. As a result, this new node will never be found during a search. A better way to handle the insertion of a duplicated key is for the value associated with that new key to replace the old value.

* **get()** retrieves the value for a given key.
<br>

* **delete()** deletes a key
    * If the tree only has a single node (root), we still must make sure the key of the root matches the key to be deleted.
    * If the tree has more than one node, use `_get()` to find the node to be removed.
        * If the key is not found, raises an error.
    * Once the node is found, there are three scenarios:
        1. The node to be deleted has no children.
        2. The node to be deleted has only one child.
        3. The node to be deleted has two children:
            * We cannot simply promote one of the children to take the node's place
            * We need to find the **successor** to replace the node to be deleted, which has the next largest key in the tree.
                * the successor is **guaranteed to have no more than one child** by BST property.
                * **findSuccessor()** and **findMin()** are to find the successor.
                * **spliceOut()** is to remove the successor.
            
<br>

* `__len__`, `__setitem__`, `__getitem__`, `__contains__`, `__delitem__` allows you to use this class like a dictionary with python build-in functions.


In [2]:
class BinarySearchTree:

    def __init__(self):
        self.root = None
        self.size = 0

    def length(self):
        return self.size

    def __len__(self):
        return self.size

    def put(self,key,val):
        if self.root:
            self._put(key,val,self.root)
        else:
            self.root = TreeNode(key,val)
        self.size = self.size + 1

    def _put(self,key,val,currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                   self._put(key,val,currentNode.leftChild)
            else:
                   currentNode.leftChild = TreeNode(key,val,parent=currentNode)
        else:
            if currentNode.hasRightChild():
                   self._put(key,val,currentNode.rightChild)
            else:
                   currentNode.rightChild = TreeNode(key,val,parent=currentNode)

    def __setitem__(self,k,v):
        self.put(k,v)

    def get(self,key):
        if self.root:
            res = self._get(key,self.root)
            if res:
                return res.payload
            else:
                return None
        else:
            return None

    def _get(self,key,currentNode):
        if not currentNode:
            return None
        elif currentNode.key == key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key,currentNode.leftChild)
        else:
            return self._get(key,currentNode.rightChild)

    def __getitem__(self,key):
        return self.get(key)

    def __contains__(self,key):
        if self._get(key,self.root):
            return True
        else:
            return False

    def delete(self,key):
        if self.size > 1:
            nodeToRemove = self._get(key,self.root)
            if nodeToRemove:
                self.remove(nodeToRemove)
                self.size = self.size-1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    def __delitem__(self,key):
        self.delete(key)

    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.hasAnyChildren():
            if self.hasLeftChild():
                if self.isLeftChild():
                    self.parent.leftChild = self.leftChild
                else:
                    self.parent.rightChild = self.leftChild
                    self.leftChild.parent = self.parent
        else:     
            if self.isLeftChild():
                self.parent.leftChild = self.rightChild
            else:
                self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent

    def findSuccessor(self):
        succ = None
        if self.hasRightChild():
            succ = self.rightChild.findMin()
        else:
            if self.parent:
                if self.isLeftChild():
                    succ = self.parent
                else:
                    self.parent.rightChild = None
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ

    def findMin(self):
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current

    def remove(self,currentNode):
        if currentNode.isLeaf(): #leaf
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        elif currentNode.hasBothChildren(): #interior
            succ = currentNode.findSuccessor()
            succ.spliceOut()
            currentNode.key = succ.key
            currentNode.payload = succ.payload

        else: # this node has one child
            if currentNode.hasLeftChild():
                if currentNode.isLeftChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild
                elif currentNode.isRightChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild
                else:
                    currentNode.replaceNodeData(currentNode.leftChild.key,
                                    currentNode.leftChild.payload,
                                    currentNode.leftChild.leftChild,
                                    currentNode.leftChild.rightChild)
            else:
                if currentNode.isLeftChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild
                elif currentNode.isRightChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                else:
                    currentNode.replaceNodeData(currentNode.rightChild.key,
                                    currentNode.rightChild.payload,
                                    currentNode.rightChild.leftChild,
                                    currentNode.rightChild.rightChild)



In [3]:
bst = BinarySearchTree()

bst.put(3, 'C')
bst.put(1, 'A')
bst.put(2, 'B')
bst.put(5, 'F')
bst.put(4, 'D')
bst.put(6, 'E')

In [4]:
len(bst)

6

In [ ]:
# the iteration part has some bug - to be fixed in future
for r in bst:
    print(r)

## References
* [(2019 Jose) Trees](https://www.udemy.com/course/python-for-data-structures-algorithms-and-interviews/)
* [Wikipedia: Binary Search Tree](https://en.wikipedia.org/wiki/Binary_search_tree)